#### Примапим диск

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Устанавливаем зависимости

In [2]:
repo_folder = '/content/drive/MyDrive/github/'

In [3]:
reqs_path = repo_folder + 'IntelligentDocumentProcessing/requirements.txt '
!pip3 install -r {reqs_path}

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 28.9 MB/s 
     |████████████████████████████████| 604 kB 66.1 MB/s 
     |████████████████████████████████| 707 kB 60.5 MB/s 
     |████████████████████████████████| 97 kB 8.4 MB/s 
     |████████████████████████████████| 49.9 MB 1.8 MB/s 
     |████████████████████████████████| 50 kB 8.9 MB/s 
     |████████████████████████████████| 5.3 MB 47.2 MB/s 
     |████████████████████████████████| 101 kB 14.4 MB/s 
     |████████████████████████████████| 175 kB 73.3 MB/s 
     |████████████████████████████████| 182 kB 73.2 MB/s 
     |████████████████████████████████| 168 kB 69.0 MB/s 
     |████████████████████████████████| 529 kB 62.4 MB/s 
     |████████████████████████████████| 2.1 MB 58.6 MB/s 
     |████████████████████████████████| 7.6 MB 52.8 MB/s 
     |████████████████████████████████| 182 kB 75.1 MB/s 
     |█████████████████████████████

In [4]:
import sys
base_folder = repo_folder + 'IntelligentDocumentProcessing/Resources/d_Named_Entity_Recognition/'
sys.path.append(base_folder)

In [5]:
import json
from tqdm import tqdm
from typing import List, Tuple
from ipymarkup import show_span_line_markup

# Как хранить разметку для NER?

### ConLL
Как исследователь данных вы должны быть знакомы с различными общими и соревнованием ConLL. В 2003 г. было опубликовано одно из первых "состязаний" по NER, включающим данные по форммату ConLL. Подробное описание вы можете найти [здесь](https://aclanthology.org/W03-0419.pdf). CoNLL — условное название форматов TSV для задач NLP (TSV — значения разделенные табуляцией, т. е. CSV с <TAB> в качестве разделителя)


```
-DOCSTART- -X- -X- O

EU NNP B-NP B-ORG
rejects VBZ B-VP O
German JJ B-NP B-MISC
call NN I-NP O
to TO B-VP O
boycott VB I-VP O
British JJ B-NP B-MISC
lamb NN I-NP O
. . O O

Peter NNP B-NP B-PER
Blackburn NNP I-NP I-PER
```

В форматах **CoNLL**:

* каждое слово (лексема) представлено в отдельной строке
* каждое предложение отделяется от следующего пустой строкой
* каждый столбец представляет собой одну аннотацию
* каждое слово в предложении имеет одинаковое количество столбцов, то есть аннотации
* аннотация - строковое значение определнной лингвистической каетегории
* аннотации, охватывающие несколько слов, иногда используют специальные обозначения, например, круглые скобки (указывающие начало и конец фразы), также может быть тег схемы IOBES (например, B-NP (Begin - Noun Phrase): начальный токен именной группы, I-NP: один из неначальных токенов именной группы , E-NP: конечный токен именной группы, S-NP: именная группа состоит из одного токена, O: не относится к именной группе)
* некоторые форматы CoNLL имеют один или несколько столбцов с числовыми идентификаторами, в таком случае следующий столбец после него содержит строковое значение - сам токен


Существуют разные форматы **ConLL**? Да.

Примерами этого разнообразия являются дорожки в соревнования ConLL-2009, а также всемирный NLP проект: [Universal Dependacies](https://universaldependencies.org/) - фреймворк для создания и хранения консистентной грамматической аннотации  (части речи, морфологические признаки, и синтаксические зависимости) на разных языках. UD — это открытое сообщество, в котором более 300 участников, которые создали около 200 наборов синтаксических деревьев на более чем 100 языках. 



In [6]:
!wget https://data.deepai.org/conll2003.zip 
!unzip conll2003.zip

--2022-11-16 20:27:49--  https://data.deepai.org/conll2003.zip
Resolving data.deepai.org (data.deepai.org)... 5.9.140.253
Connecting to data.deepai.org (data.deepai.org)|5.9.140.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 982975 (960K) [application/x-zip-compressed]
Saving to: ‘conll2003.zip’

conll2003.zip       100%[===================>] 959.94K  2.03MB/s    in 0.5s    

2022-11-16 20:27:50 (2.03 MB/s) - ‘conll2003.zip’ saved [982975/982975]

Archive:  conll2003.zip
  inflating: metadata                
  inflating: test.txt                
  inflating: train.txt               
  inflating: valid.txt               


От теории к практике. Давайте прочитаем сохраненную аннотация и стркутрируем ее в виде исходных документов и предложений внутри. Мы ожидаем получить список документов, каждый из которых будет состоять из списка предложений. Внутри предложения будут пары: токен - NER тег.

In [7]:
_SEP = " "
_TOKEN_COLUMN = 0
_LABEL_COLUMN = -1
_CONLL_PREFIX_SEPARATOR = "-"
_LIBRARY_PREFIX_SEPARATOR = "_"

def read_conll(file_path: str) -> List[List[Tuple[List[str], List[str]]]]:

  with open(file_path) as f:
    lines = f.readlines()

  documents = []
  document_sequences = []
  sentence_tokens = []
  sentence_labels = []
  # не забудем tqdm для динамического отслеживания процесс считывания разметки
  for line in tqdm(lines, desc="Parsing data"):
      contents = line.strip()
      # каждый новый документ начинается с этого тега
      if contents.startswith("-DOCSTART-"):
          if document_sequences:
              documents.append(document_sequences)
          document_sequences = []
          sentence_tokens = []
          sentence_labels = []
          continue
      if not contents:
          if sentence_tokens:
              document_sequences.append((sentence_tokens, sentence_labels))
          sentence_tokens = []
          sentence_labels = []
          continue
      items = contents.split()
      token = items[_TOKEN_COLUMN]
      label = items[_LABEL_COLUMN].replace(
                _CONLL_PREFIX_SEPARATOR, _LIBRARY_PREFIX_SEPARATOR
            )
      sentence_tokens.append(token)
      sentence_labels.append(label)
    
  if sentence_tokens:
    document_sequences.append((sentence_tokens, sentence_labels))

  if document_sequences:
    documents.append(document_sequences)

  return documents

In [8]:
train_docs = read_conll("train.txt")

Parsing data: 100%|██████████| 219554/219554 [00:00<00:00, 790590.13it/s]


Посмотрим как выглядят выравненные предложения и списки меток:

In [9]:
doc_example_id = 8
print("START DOC\n" + "-" * 30)
for sentence, labels in train_docs[doc_example_id]:
  print(" ".join(sentence))
  print(" ".join(labels))
print("-" * 30 + "\nEND DOC")

START DOC
------------------------------
Port conditions update - Syria - Lloyds Shipping .
O O O O B_LOC O B_ORG I_ORG O
Port conditions from Lloyds Shipping Intelligence Service --
O O O B_ORG I_ORG I_ORG I_ORG O
LATTAKIA , Aug 10 - waiting time at Lattakia and Tartous presently 24 hours .
B_LOC O O O O O O O B_LOC O B_LOC O O O O
------------------------------
END DOC


В коде ниже мы преобразуем наш дадасет: соберем через пробелы токены и воостановим в них координаты сущностей в символах. Таким образом мы будем иметь на каждое предложение список туплов для сущностей и одну строку, содержащую строковое значение.

In [10]:
def document_to_samples(
        doc_id: int,
        document_sequences: List[Tuple[List[str], List[str]]],
):
    """
    На вход получаем список предложений с тегами для каждого токена внутри предложения. 
    В данном пайплайне предложения относятся к одному документу. Исходные токены 
    предложения собираем в одну строку через пробел. Преобразуем список тегов 
    сущностей к списку туплов с символьными координамтами и типом (для каждой сущности).
    """
    instances = []
    sentences = []
    sequence_boundaries = []
    current_sentence_pos = 0
    for sentence_tokens, sentence_labels in document_sequences:
        current_word_pos = 0
        sentence_text = _SEP.join(sentence_tokens)
        sentences.append(sentence_text)
        sequence_boundaries.append(
            (current_sentence_pos, current_sentence_pos + len(sentence_text))
        )
        prev_entity_type = ""
        entity_type = ""
        entities = []
        current_sentence_pos = 0
        token_boundaries = []
        entity_start = 0
        entity_end = 1
        entity_text = []
        for i, (tag, word) in enumerate(zip(sentence_labels, sentence_tokens)):
            token_boundaries.append((current_word_pos, current_word_pos + len(word)))
            current_word_pos += len(word) + len(_SEP)

            if tag != "O":
                prefix, curr_entity_type = tag.split(_LIBRARY_PREFIX_SEPARATOR, 1)
            else:
                prefix = ''
                curr_entity_type = ''

            if (prefix not in ("B", "I")) and ((prefix == '') and tag != "O"):
                continue

            if prefix == "I" and prev_entity_type == curr_entity_type:
                entity_end += 1
                entity_text.append(word)

            else:
                if entity_type:
                    entities.append(
                        (token_boundaries[entity_start][0], 
                         token_boundaries[entity_end - 1][-1], 
                         entity_type)
                        )
                entity_start = i
                entity_end = i + 1
                entity_type = curr_entity_type
                entity_text = [word]

            prev_entity_type = curr_entity_type

        else:
            if entity_type:
                entities.append(
                        (token_boundaries[entity_start][0], 
                         token_boundaries[entity_end - 1][-1], 
                         entity_type))

        instance = (sentence_text, entities)
        instances.append(instance)
        current_sentence_pos += len(sentence_text) + len(_SEP)
    return instances

def documents_to_samples(
  samples: List[List[Tuple[List[str], List[str]]]]
) -> List[Tuple[str, List[Tuple[int, int, str]]]]:
  """
  Главаня функция для преобразования примеров документов из исходного 
  формата к формату визуализации.
  Уходим от уровня документов на уровень предложений.
  """
  parser_result = []
  for doc_id, document_sequences in enumerate(samples):
      parser_result.extend(
          document_to_samples(doc_id, document_sequences)
      )
  return parser_result

In [11]:
train_sentences = documents_to_samples(train_docs)

In [12]:
train_sentences[0]

('EU rejects German call to boycott British lamb .',
 [(0, 2, 'ORG'), (11, 17, 'MISC'), (34, 41, 'MISC')])

А теперь можем визулизировать первые 5 предложений:

In [13]:
visualize_n_example = 5
for i in train_sentences[:visualize_n_example]:
  show_span_line_markup(*i)
  print()

## Label studio

Согласно главной странице инструмента [**Label studio**](https://labelstud.io/) - "это самый гибкий инстурмент для аннотирования данных". Вы сможете конфигурировать кастомные шаблоны для построения процесса разметки, а если вы новичок, то достаточно будет воспользоваться готовыми темплейтами. Установка не потребует большого количества времени.



Действительно, с помощью готовых шаблонов мы сможете решать найиболее частотные NLP задачи:

1.   *Классификация* : Классификация документов мультиклассовая/мультилейбловая с набором до 1000 заданных типов

2.   *Извлечение сущностей*: Сегментация и категоризация фрагментов текста

3. *Вопросно-ответные системы*: Разметка для построения датасета вопросно-ответной системы с опорой на контекст 

4. *Анализ тональности*: Определение типа тональности текста (до N категорий)


#### Задача #1
В **Label studio** было размечено два документа с помощью шаблона "Извлечение сущностей". Ваша задача считать разметку из файла-выгрузки и преобразовать ее к формату пригодному для визулизации с помощью **ipymarkup**. Ожидаемые типы сущностей **LOC**, **ORG**, **PER**.

In [14]:
! head {base_folder}/resources/is_markup/project-770-at-2022-04-03-15-42-e657ff5c.json

[{"id":251710,"annotations":[{"id":142494,"completed_by":57,"result":[{"id":"uoVEuYDPxP","type":"labels","value":{"end":203,"text":"Евгений Красников","start":186,"labels":["PER"]},"origin":"manual","to_name":"text","from_name":"label"},{"id":"803QOHrY4b","type":"labels","value":{"end":58,"text":"«Зенита»","start":50,"labels":["ORG"]},"origin":"manual","to_name":"text","from_name":"label"},{"id":"L_TMhypNgJ","type":"labels","value":{"end":87,"text":"«Металлиста»","start":75,"labels":["ORG"]},"origin":"manual","to_name":"text","from_name":"label"},{"id":"CWMau-ScoY","type":"labels","value":{"end":143,"text":"«Зенитом»","start":134,"labels":["ORG"]},"origin":"manual","to_name":"text","from_name":"label"},{"id":"V6Y2BmJ1wJ","type":"labels","value":{"end":299,"text":"«Зенита»","start":291,"labels":["ORG"]},"origin":"manual","to_name":"text","from_name":"label"},{"id":"fk6W4Ug38H","type":"labels","value":{"end":624,"text":"«Зенита»","start":616,"labels":["ORG"]},"origin":"manual","to_name":

In [52]:
def read_ls_data(file_path: str) -> List[Tuple[str, List[Tuple[int, int, str]]]]:
  """
  Your code here
  """
  with open(file_path, 'r') as f:
    data = json.load(f)

  samples = []
  doc_text = []
  for d in data:
    samples.append([])
    doc_text.append(d['data']['text'])
    for res in d['annotations']:
      for val in res['result']:
        samples[-1].append((val['value']['start'], val['value']['end'], val['value']['labels'][0]))
  return list(zip(doc_text, samples))

In [53]:
file_path = "/content/drive/MyDrive/github/IntelligentDocumentProcessing/Resources/d_Named_Entity_Recognition/resources/is_markup/project-770-at-2022-04-03-15-42-e657ff5c.json"
result = read_ls_data(file_path)

#### Проверить решение

In [54]:
assert len(result) == 2, "Должно быть размечено два документа"
assert all([True for doc_text, markup in result if markup]), "В каждом документе должна быть разметка"

expected_markup = [(29, 44, 'PER'), (56, 69, 'ORG'), (327, 331, 'LOC'), (367, 375, 'ORG')]

_, result_markup = result[1]
result_markup = sorted(result_markup, key=lambda x: x[0])
for res_m, expect_m in zip(result_markup, expected_markup):
  assert res_m == expect_m, f"Полученная и ожидаемая разметка не совпадают {res_m} != {expect_m}"


print("Все в порядке :)")

Все в порядке :)


In [55]:
show_span_line_markup(*result[0])